# Description

Examine the asymptotic behavior of the Eyring-Kramers formula constants (energy barrier and prefactor) in the nonlocal case ($k>1$), varying the number of sites, $n$.  This requires running string method using, for instance, `nonlocal.jl` from the `scripts` folder.

In [ ]:
using Plots
using Printf
using Random
using LaTeXStrings
using LinearAlgebra
using ForwardDiff
using JLD2

In [ ]:
default(xtickfontsize=14, ytickfontsize=14,
    guidefontsize=14, legendfontsize=12, lw=2, ms=8)

In [ ]:
push!(LOAD_PATH, "../TwistedStates");
push!(LOAD_PATH, "../ClassicalKuramoto");

In [ ]:
using GraphMatrices
using TwistedStates
using ClassicalKuramoto

# Set Parameters

In [ ]:
k = 2; # k > 1 is nonlocal coupling
α = 0;
q_vals = [1, 2, 3, 4]; # twisted states to examine
E = u -> ClassicalKuramoto.energy(u, k, α);
HessE(u) = ForwardDiff.hessian(E, u);

# Load Data

In [ ]:
filepath = "/Users/grs53/Library/CloudStorage/OneDrive-DrexelUniversity/simulation_data/kuramoto/nonlocal";
filenames = ["nonlocal2_q1_k2_nmin10_nmax1280_dt0_01_N100.jld2",
    "nonlocal2_q2_k2_nmin20_nmax1280_dt0_01_N100.jld2",
    "nonlocal2_q3_k2_nmin40_nmax1280_dt0_01_N100.jld2",
    "nonlocal2_q4_k2_nmin40_nmax1280_dt0_01_N200.jld2"]


n_vals=[];
saddle_states=[];
string_paths = [];

for name in filenames
    data = jldopen(filepath * "/" * name);
    push!(n_vals, deepcopy(data["n_vals"]))
    push!(saddle_states, deepcopy(data["saddle_states"]))
    push!(string_paths, deepcopy(data["string_paths"]))
end

# Compute Constants

In [ ]:
ΔE_vals = [];
C_vals = [];
for q in q_vals
    @show q;
    ΔE_vals_ = Float64[]
    C_vals_ = Float64[];
    for (n, saddle) in zip(n_vals[q], saddle_states[q])
        u_left = construct_q_twisted(n, q)
        @show n
        @show ΔE = E(saddle) - E(u_left)
        evals_min = eigvals(HessE(u_left))
        nonzero_evals_min = evals_min[2:end]
        evals_saddle = eigvals(HessE(saddle))
        nonzero_evals_saddle = [evals_saddle[1]; evals_saddle[3:end]]
        C_ = (2 * π) / (abs(nonzero_evals_saddle[1])) * sqrt(prod(abs.(nonzero_evals_saddle) ./ (nonzero_evals_min)))
        push!(ΔE_vals_, ΔE)
        push!(C_vals_, C_)
    end
    push!(ΔE_vals, deepcopy(ΔE_vals_))
    push!(C_vals, deepcopy(C_vals_))
end

In [ ]:
markers = [:circle, :rect, :diamond, :hexagon]

In [ ]:
q = 1;
scatter(n_vals[q], ΔE_vals[q], label=latexstring(@sprintf("q = %d", q - 1)), marker=markers[q],xscale=:log10)
for q in q_vals[2:end]
    scatter!(n_vals[q], ΔE_vals[q], label=latexstring(@sprintf("q = %d", q - 1)), marker=markers[q])
end
xlabel!(L"n")
ylabel!(L"H_{q+1}")
title!(latexstring(@sprintf("k = %d", k)))


In [ ]:
q = 1;
scatter(n_vals[q], kramer_vals[q], label=latexstring(@sprintf("q = %d", q - 1)), marker=markers[q], xscale=:log10)
for q in q_vals[2:end]
    scatter!(n_vals[q], kramer_vals[q], label=latexstring(@sprintf("q = %d", q - 1)), marker=markers[q])
end

xlabel!(L"n")
ylabel!(L"n C(n,q)")
title!(latexstring(@sprintf("k = %d", k)))
